In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
%matplotlib inline 
%load_ext memory_profiler
from sklearn.metrics import make_scorer
from scipy.special import expit
import time
import math
import random
from memory_profiler import memory_usage
from sklearn import metrics as mt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold



from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

target_classifier = 'PC'
df = pd.read_csv('responses.csv', sep=",")

In [6]:
# remove rows whose target classfier value is NaN
df_cleaned_classifier = df[np.isfinite(df[target_classifier])]
# change NaN number values to the mean
df_imputed = df_cleaned_classifier.fillna(df.mean())
# get categorical features
object_features = list(df_cleaned_classifier.select_dtypes(include=['object']).columns)
# one hot encode categorical features
one_hot_df = pd.concat([pd.get_dummies(df_imputed[col],prefix=col) for col in object_features], axis=1)
# drop object features from imputed dataframe
df_imputed_dropped = df_imputed.drop(object_features, 1)
frames = [df_imputed_dropped, one_hot_df]
# concatenate both frames by columns
df_fixed = pd.concat(frames, axis=1)

In [7]:
# Research on Cost Matrix
# http://www.ibm.com/support/knowledgecenter/SSEPGG_11.1.0/com.ibm.im.model.doc/c_cost_matrix.html

cost_matrix = np.matrix([[0,1,2,3,4],
[1,0,1,2,3],
[3,1,0,1,2],
[5,3,1,0,1],
[7,5,2,1,0]])

def get_confusion_costTot(confusion_matrix, cost_matrix):
    score = np.sum(confusion_matrix*cost_matrix)
    return score

confusion_scorer = make_scorer(get_confusion_costTot, greater_is_better=False)
confusion_scorer

make_scorer(get_confusion_costTot, greater_is_better=False)

In [8]:
from sklearn.model_selection import ShuffleSplit

# we want to predict the X and y data as follows:
if target_classifier in df_fixed:
    y = df_fixed[target_classifier].values # get the labels we want
    del df_fixed[target_classifier] # get rid of the class label
    X = df_fixed.values # use everything else to predict!

X = X/5
num_folds = 10

cv_object = StratifiedKFold(n_splits= num_folds, random_state=None, shuffle=True)
cv_object.split(X,y)

print(cv_object)

StratifiedKFold(n_splits=10, random_state=None, shuffle=True)


In [9]:
for train_indices, test_indices in cv_object.split(X,y): 
        # I will create new variables here so that it is more obvious what 
        # the code is doing (you can compact this syntax and avoid duplicating memory,
        # but it makes this code less readable)
        X_train = (X[train_indices])
        y_train = y[train_indices]

    #     print(X_train)
    #     print(y_train)

        X_test = (X[test_indices])
        y_test = y[test_indices]

In [39]:


perclass_mean_tpr = 0.0
roc_auc = 0
classes = np.unique(y[train_indices])
# get the mean fpr and tpr, per class
for j in classes:
    fpr, tpr, thresholds = roc_curve(y[test_indices],
    y_hat,
    pos_label=j)
    perclass_mean_tpr += interp(mean_fpr, fpr, tpr)
    perclass_mean_tpr[0] = 0.0
    roc_auc += auc(fpr, tpr)

perclass_mean_tpr /= len(classes)
roc_auc /= len(classes)
mean_tpr += perclass_mean_tpr
plt.plot(mean_fpr,perclass_mean_tpr,'--',lw=1,label='Mean Class ROC fold %d (area = %0.2f)'
% (i+1, roc_auc))
mean_tpr /= K
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr,perclass_mean_tpr,'k-',lw=2,label='Total Mean ROC (area = %0.2f)'
% (mean_auc))
plt.legend(loc='best')
plt.grid()

NameError: name 'roc_curve' is not defined

In [31]:



clf = MLPClassifier()
    



In [32]:
from sklearn.ensemble import BaggingClassifier

class MyEnsemble():
    
    def __init__(self, c, num_c, max_s, v):
        self.Ensemble = BaggingClassifier(base_estimator= c,
                                    n_estimators = num_c,
                                     max_samples = max_s,
                                     verbose = v)
    def predict(self, X):
        return self.Ensemble.predict(X)
    
    def fit(self, X,y):
        self.Ensemble.fit(X,y)
        
    def fit_random():
        
        
    def predict_proba(self, X):
        return self.Ensemble.predict_proba(X)

In [33]:
num_instances = 10


ensemble = MyEnsemble(clf, 10,y_train.shape[0],False)

ensemble.fit(X_train,y_train)
y_hat=ensemble.predict(X_test)
# print(y_hat)
print(mt.confusion_matrix(y_hat,y_test))

print(X_train.shape)

[[ 1  8  2  1  0]
 [ 6  7  8  3  0]
 [ 3  2  7  4  4]
 [ 2  1  6  4  5]
 [ 1  2  2  8 11]]
(906, 172)


In [34]:
ensemble.predict_proba(X_test)

array([[  2.13567473e-02,   7.80820377e-02,   1.49648584e-01,
          4.98719836e-01,   2.52192795e-01],
       [  1.68573261e-01,   3.30600520e-01,   2.53932192e-01,
          2.05113294e-01,   4.17807332e-02],
       [  3.04691360e-03,   3.26662785e-02,   5.61833446e-02,
          7.80429620e-01,   1.27673843e-01],
       [  2.43917055e-01,   3.12643492e-01,   2.49683435e-01,
          1.18149003e-01,   7.56070152e-02],
       [  4.35595902e-01,   2.50006305e-01,   2.69534934e-01,
          3.32737036e-02,   1.15891557e-02],
       [  9.45286122e-02,   1.08462859e-01,   2.64324665e-01,
          4.64878874e-01,   6.78049901e-02],
       [  3.05854270e-03,   2.42194277e-02,   6.81727207e-02,
          1.62198504e-01,   7.42350805e-01],
       [  1.37649823e-01,   4.84684934e-01,   1.39503748e-01,
          5.38579228e-02,   1.84303572e-01],
       [  4.60385181e-02,   1.75355470e-01,   3.04740852e-01,
          3.65799782e-01,   1.08065378e-01],
       [  4.23592494e-02,   1.4078171

In [35]:
 acc = mt.accuracy_score(y_test,y_hat)
            #         lr_clf_accuracies.append(acc)
            #         cost_accuracies.append([acc])

conf = mt.confusion_matrix(y_test,y_hat)
print(conf)

[[ 1  6  3  2  1]
 [ 8  7  2  1  2]
 [ 2  8  7  6  2]
 [ 1  3  4  4  8]
 [ 0  0  4  5 11]]


In [36]:
confidence = ensemble.

SyntaxError: invalid syntax (<ipython-input-36-2f506efacab3>, line 1)

In [18]:
score = get_confusion_costTot(conf, cost_matrix)

In [19]:
score

980

In [ ]:
# Great! Its seems we are also maxing out the F1 score, lets go with C==1e-3
from sklearn.metrics import roc_curve, auc
from scipy import interp

K = 4

if sklearn_version < '0.18':
    from sklearn.cross_validation import StratifiedKFold
    kfold = StratifiedKFold(y=y_train, 
                            n_folds=K,
                            random_state=1)
else:
    from sklearn.model_selection import StratifiedKFold
    kfold = StratifiedKFold(n_splits=K,
                            random_state=1).split(X_train, y_train)


mean_tpr = 0.0
mean_fpr = np.linspace(0, 1, 100)
all_tpr = []

for i, (train, test) in enumerate(kfold):
    probas = pipe_lr.fit(X_train[train],
                         y_train[train]).predict_proba(X_train[test])

    perclass_mean_tpr = 0.0
    roc_auc = 0
    classes = np.unique(y_train[train])
    # get the mean fpr and tpr, per class
    for j in classes:
        fpr, tpr, thresholds = roc_curve(y_train[test],
                                         probas[:, j],
                                         pos_label=j)
        perclass_mean_tpr += interp(mean_fpr, fpr, tpr)
        perclass_mean_tpr[0] = 0.0
        roc_auc += auc(fpr, tpr)
        
    perclass_mean_tpr /= len(classes)
    roc_auc /= len(classes)
    mean_tpr += perclass_mean_tpr
    plt.plot(mean_fpr,perclass_mean_tpr,'--',lw=1,label='Mean Class ROC fold %d (area = %0.2f)'
                   % (i+1, roc_auc))
    
mean_tpr /= K
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr,perclass_mean_tpr,'k-',lw=2,label='Total Mean ROC (area = %0.2f)'
                   % (mean_auc))
plt.legend(loc='best')
plt.grid()